In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ply
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

"dengue_features_train.csv" - split into 80 ("train") / 20 ("test")
"dengue_features_test.csv" - data for scoring on the DrivenData challende ("Hold out")

In [4]:
X = pd.read_csv('../../data/dengue_features_train.csv')
y = pd.read_csv('../../data/dengue_labels_train.csv', usecols=['total_cases'])

In [7]:
def data_preprocess(df):
    # drop or encode categorical cols
    df_processed = df.drop('week_start_date', axis=1)
    df_processed['city'] = df_processed['city'].apply(lambda x : 1 if x=='iq' else 0)
    return df_processed

In [8]:
X_processed = data_preprocess(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size = 0.20, random_state = 42, stratify = X_processed['city'])

In [35]:
n_neighbors = 2
n_estimators = 30
n_components = 2

In [21]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('imputer',KNNImputer(n_neighbors = n_neighbors)),
    ('regressor', RandomForestRegressor(random_state=42, n_estimators = n_estimators))
])

In [28]:
pipe.fit(X_train, y_train)
print('Testing score: ', pipe.score(X_test, y_test))

/home/asa/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


Testing score:  0.5597929445295612


In [29]:
y_pred = pipe.predict(X_test)
print(f'MAE = {mean_absolute_error(y_test, y_pred)}')

MAE = 12.403082191780822


## Working first pipeline!  Scores match MAE from 'modeling.ipynb'

In [36]:
pipe_2 = Pipeline([
    ('scaler', StandardScaler()),
    ('imputer',KNNImputer(n_neighbors = n_neighbors)),
    ('dim_reduction', PCA(n_components = n_components)),
    ('regressor', RandomForestRegressor(random_state=42, n_estimators = n_estimators))
])

In [37]:
pipe_2.fit(X_train, y_train)
y_pred_2 = pipe_2.predict(X_test)
print(f'MAE = {mean_absolute_error(y_test, y_pred_2)}')

MAE = 19.86187214611872


/home/asa/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
